# Azure DALL-E 이미지 생성 예제

이 노트북은 Azure OpenAI 서비스를 사용하여 이미지를 생성하는 방법을 보여줍니다.

설정 ## 설정

먼저 필요한 종속성을 설치합니다.

In [ ]:
! pip install openai
# We need requests to retrieve the generated image
! pip install requests
# We use Pillow to display the generated image
! pip install pillow 
# (Optional) If you want to use Microsoft Active Directory
! pip install azure-identity

In [ ]:
import os
import openai


또한, Azure OpenAI 서비스에 제대로 접속하기 위해서는 [Azure 포털](https://portal.azure.com)에서 적절한 리소스를 생성해야 합니다(자세한 방법은 [Microsoft Docs](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal)에서 확인 가능).

리소스가 생성되면 가장 먼저 사용해야 하는 것은 엔드포인트입니다. 엔드포인트는 *"리소스 관리"* 섹션의 *"키 및 엔드포인트"* 섹션을 확인하면 얻을 수 있습니다. 이 정보가 있으면 이 정보를 사용하여 SDK를 설정합니다:

In [ ]:
openai.api_base = '' # Add your endpoint here

# At the moment DALL·E is only supported by the 2023-06-01-preview API version
openai.api_version = '2023-06-01-preview'

인증 ### 인증

Azure OpenAI 서비스는 API 키 및 Azure 자격 증명을 포함하는 여러 인증 메커니즘을 지원합니다.

In [ ]:
use_azure_active_directory = False


#### API 키를 사용한 인증

OpenAI SDK가 *Azure API 키*를 사용하도록 설정하려면 `api_type`을 `azure`로 설정하고 `api_key`를 엔드포인트와 연결된 키로 설정해야 합니다(이 키는 [Azure 포털](https://portal.azure.com)의 *"리소스 관리"* 아래 "키 및 엔드포인트"*에서 찾을 수 있음).

In [ ]:
if not use_azure_active_directory:
    openai.api_type = 'azure'
    openai.api_key = os.environ["OPENAI_API_KEY"]

> 참고: 이 예제에서는 코드에서 변수를 설정하여 Azure API를 사용하도록 라이브러리를 구성했습니다. 개발의 경우 환경 변수를 대신 설정하는 것을 고려하세요:

```
OPENAI_API_BASE
OPENAI_API_KEY
OPENAI_API_TYPE
OPENAI_API_VERSION
```

#### Microsoft Active Directory를 사용한 인증
이제 Microsoft Active Directory 인증을 통해 키를 얻는 방법을 살펴보겠습니다.

In [ ]:
from azure.identity import DefaultAzureCredential

if use_azure_active_directory:
    default_credential = DefaultAzureCredential()
    token = default_credential.get_token("https://cognitiveservices.azure.com/.default")

    openai.api_type = 'azure_ad'
    openai.api_key = token.token

토큰은 일정 기간 동안 유효하며 그 이후에는 만료됩니다. 모든 요청에 유효한 토큰이 전송되도록 하려면 요청.auth에 연결하여 만료되는 토큰을 새로 고칠 수 있습니다:

In [ ]:
import typing
import time
import requests

if typing.TYPE_CHECKING:
    from azure.core.credentials import TokenCredential

class TokenRefresh(requests.auth.AuthBase):

    def __init__(self, credential: "TokenCredential", scopes: typing.List[str]) -> None:
        self.credential = credential
        self.scopes = scopes
        self.cached_token: typing.Optional[str] = None

    def __call__(self, req):
        if not self.cached_token or self.cached_token.expires_on - time.time() < 300:
            self.cached_token = self.credential.get_token(*self.scopes)
        req.headers["Authorization"] = f"Bearer {self.cached_token.token}"
        return req

if use_azure_active_directory:
    session = requests.Session()
    session.auth = TokenRefresh(default_credential, ["https://cognitiveservices.azure.com/.default"])

    openai.requestssession = session

세대 ## 세대

설정 및 인증이 완료되면 이제 Azure OpenAI 서비스에서 이미지를 생성하고 반환된 URL에서 이미지를 검색할 수 있습니다.

#### 1. 이미지 생성하기

이 프로세스의 첫 번째 단계는 실제로 이미지를 생성하는 것입니다:

In [ ]:
generation_response = openai.Image.create(
    prompt='A cyberpunk monkey hacker dreaming of a beautiful bunch of bananas, digital art',
    size='1024x1024',
    n=2
)

print(generation_response)

Image.create` 호출의 응답이 있으면 `requests`를 사용하여 URL에서 다운로드합니다.

In [ ]:
import os
import requests

# First a little setup
image_dir = os.path.join(os.curdir, 'images')
# If the directory doesn't exist, create it
if not os.path.isdir(image_dir):
    os.mkdir(image_dir)

# With the directory in place, we can initialize the image path (note that filetype should be png)
image_path = os.path.join(image_dir, 'generated_image.png')

# Now we can retrieve the generated image
image_url = generation_response["data"][0]["url"]  # extract image URL from response
generated_image = requests.get(image_url).content  # download the image
with open(image_path, "wb") as image_file:
    image_file.write(generated_image)

이미지를 다운로드한 후 [Pillow](https://pypi.org/project/Pillow/) 라이브러리를 사용하여 이미지를 열고 표시합니다:

In [ ]:
from PIL import Image 

display(Image.open(image_path))